In [1]:
from moleculekit.molecule import Molecule
import os
import importlib
from torchmdexp.metrics.rmsd import rmsd
from torchmdexp.metrics.l2_rmsd import l2_rmsd
from torchmdexp.metrics.log_rmsd import log_rmsd
from torchmdexp.metrics.tmscore import tm_score
from torchmdexp.samplers.utils import get_native_coords, get_embeddings
from moleculekit.projections.metricrmsd import MetricRmsd
from moleculekit.projections.metrictmscore import MetricTMscore
import numpy as np
import torch
from torchmdexp.metrics.native_contacts import q

device = "cuda:1"
precision = torch.float

In [2]:
ref_mol = Molecule('/shared/carles/repo/torchmd-exp/data/ww/ww_ca.pdb')
ref_mol.read('/shared/carles/repo/torchmd-exp/data/ww/ww_ca_top_dih.psf')

mol = ref_mol

In [3]:
native_coords = get_native_coords(ref_mol, device)
native_coords = native_coords.type(precision)
coords = get_native_coords(mol, device)
coords = coords.type(precision)


print('Starting rmsd: ', rmsd(coords, native_coords), 'Starting log_rmsd: ', log_rmsd(coords, native_coords))
print('Starting tmscore: ', tm_score(coords, native_coords), 'Starting l2_rmsd: ', l2_rmsd(coords, native_coords))

Starting rmsd:  tensor(0.0037, device='cuda:1') Starting log_rmsd:  tensor(0.0036, device='cuda:1')
Starting tmscore:  tensor(1.0000, device='cuda:1') Starting l2_rmsd:  tensor(0.4963, device='cuda:1')


In [17]:
from torchmd.forcefields.forcefield import ForceField
from torchmd.parameters import Parameters


ff = ForceField.create(mol, '/shared/carles/repo/torchmd-exp/data/ca_priors-dihedrals_general_5xweaker.yaml')
parameters = Parameters(ff, mol, precision=precision, device=device)

In [18]:
from torchmd.integrator import maxwell_boltzmann
from torchmd.systems import System

system = System(mol.numAtoms, nreplicas=1, precision=precision, device=device)
system.set_positions(mol.coords)
system.set_box(mol.box)
system.set_velocities(maxwell_boltzmann(parameters.masses, T=350, replicas=1))

In [19]:
get_embeddings(mol, 'cpu', 1)

tensor([[19, 20,  8,  8,  2,  7,  6, 19, 18, 15, 12, 18, 12, 12,  2, 18, 14,  4,
          4,  3,  9, 11, 21, 13,  9,  1, 12, 10,  7,  6, 18,  8, 12,  2]])

In [20]:
externalmodule = importlib.import_module('torchmdnet.calculators')
#nnp = '/shared/carles/torchmd-exp/scripts/trainings/comparison/rmsd/betas/noise/2_layers_9_cutoff_400steps_limit3'
#nnp += '/epoch=287-train_loss=2.2491-val_loss=2.6622.ckpt'

nnp = '/shared/carles/torchmd-exp/scripts/trainings/comparison/rmsd/bbl/2_layers_9_cutoff/'
nnp += 'epoch=437-train_loss=0.6593-val_loss=1.4174.ckpt'
embeddings = torch.tensor([19, 20,  8,  8,  2,  7,  6, 19, 18, 15, 12, 18, 12, 12,  2, 18, 14,  4,
          4,  3,  9, 11, 21, 13,  9,  1, 12, 10,  7,  6, 18,  8, 12,  2]).repeat(1, 1)
external = externalmodule.External(nnp, embeddings, device)

## Compute non modified things

In [21]:
from torchmd.forces import Forces
#external=None
priors = ['bonds', 'repulsioncg', 'dihedrals']
forces = Forces(parameters,terms=priors, cutoff=9, rfa=False, switch_dist=None, 
                external=external, exclusions = ('bonds')
               )
# Evaluate current energy and forces. Forces are modified in-place
Epot = forces.compute(system.pos, system.box, system.forces, returnDetails=False)
print(Epot)
print(system.forces)

[152.59977984428406]
tensor([[[ 6.7218,  1.5067,  5.7858],
         [-0.2933, -3.6147, -5.8815],
         [-5.3660,  5.8629,  1.1280],
         [ 1.1177,  7.8655,  2.4290],
         [-0.2801, -8.6959, -3.8507],
         [ 0.3246,  0.1971,  3.8866],
         [-0.8731, -3.1763, -0.4418],
         [ 2.5349,  2.9720,  0.8734],
         [-2.6751,  5.0233,  4.3055],
         [ 5.0173, -3.7575, -0.3397],
         [-4.4940,  1.5418, -5.5471],
         [-1.9750, -2.3157,  3.9550],
         [ 1.0775, -6.8574, -0.6831],
         [-3.3446, -1.4422, -2.2691],
         [ 4.5015,  3.8879,  0.3167],
         [-0.6324, -2.4136, -1.2780],
         [ 5.4973,  0.6497,  1.3289],
         [-0.3193, -3.0437, -4.0570],
         [ 2.5220,  2.8889,  2.3606],
         [-2.8744, -3.0808,  0.1689],
         [-1.1798,  1.2846, -2.8190],
         [ 2.2826, -3.4899,  2.4782],
         [-3.2264,  1.3754,  2.1837],
         [-2.6728,  1.7922, -0.7993],
         [ 0.6755,  0.7320, -1.4132],
         [-3.5311,  0.0391,  

# Translation invariance

In [120]:
system.pos = system.pos + torch.tensor([1, 2, 3], device=device)
Epot = forces.compute(system.pos, system.box, system.forces, returnDetails=False)
print(Epot)
print(system.forces)

[152.59974241256714]
tensor([[[ 6.7218,  1.5067,  5.7858],
         [-0.2933, -3.6147, -5.8815],
         [-5.3660,  5.8629,  1.1280],
         [ 1.1177,  7.8655,  2.4290],
         [-0.2801, -8.6959, -3.8507],
         [ 0.3246,  0.1971,  3.8866],
         [-0.8731, -3.1763, -0.4418],
         [ 2.5349,  2.9720,  0.8734],
         [-2.6751,  5.0233,  4.3055],
         [ 5.0173, -3.7575, -0.3397],
         [-4.4940,  1.5418, -5.5471],
         [-1.9750, -2.3157,  3.9550],
         [ 1.0775, -6.8574, -0.6831],
         [-3.3446, -1.4422, -2.2691],
         [ 4.5015,  3.8879,  0.3167],
         [-0.6324, -2.4136, -1.2780],
         [ 5.4973,  0.6497,  1.3289],
         [-0.3193, -3.0437, -4.0570],
         [ 2.5220,  2.8889,  2.3606],
         [-2.8744, -3.0808,  0.1689],
         [-1.1798,  1.2846, -2.8190],
         [ 2.2827, -3.4900,  2.4782],
         [-3.2265,  1.3755,  2.1837],
         [-2.6728,  1.7922, -0.7993],
         [ 0.6755,  0.7320, -1.4132],
         [-3.5311,  0.0391,  

# Rotation equivariance

In [9]:
import scipy.spatial.transform as trans

# rotate around x coord by 45 degrees and comput forces
rot = trans.Rotation.from_euler("x", 32, degrees=True)


In [128]:
# Compute forces and then rotate
Epot = forces.compute(system.pos, system.box, system.forces, returnDetails=False)
print(Epot)
print(system.forces)


[152.59752750396729]
tensor([[[ 6.7227, -1.7885,  5.7060],
         [-0.2942,  0.0510, -6.9045],
         [-5.3663,  4.3742,  4.0634],
         [ 1.1180,  5.3831,  6.2277],
         [-0.2811, -5.3341, -7.8733],
         [ 0.3255, -1.8928,  3.3999],
         [-0.8745, -2.4584, -2.0583],
         [ 2.5340,  2.0581,  2.3155],
         [-2.6746,  1.9776,  6.3117],
         [ 5.0172, -3.0064, -2.2779],
         [-4.4944,  4.2473, -3.8871],
         [-1.9748, -4.0603,  2.1264],
         [ 1.0768, -5.4543, -4.2131],
         [-3.3450, -0.0203, -2.6887],
         [ 4.5023,  3.1301,  2.3290],
         [-0.6323, -1.3698, -2.3623],
         [ 5.4985, -0.1534,  1.4702],
         [-0.3198, -0.4296, -5.0530],
         [ 2.5215,  1.1988,  3.5334],
         [-2.8734, -2.7017, -1.4892],
         [-1.1796,  2.5824, -1.7094],
         [ 2.2835, -4.2729,  0.2526],
         [-3.2254,  0.0094,  2.5807],
         [-2.6734,  1.9439,  0.2712],
         [ 0.6746,  1.3695, -0.8102],
         [-3.5314,  0.0086,  

In [126]:
torch.tensor(rot.apply(system.forces[0].cpu())).to(device)

tensor([[ 6.7218, -1.7882,  5.7051],
        [-0.2933,  0.0512, -6.9033],
        [-5.3660,  4.3743,  4.0635],
        [ 1.1177,  5.3831,  6.2280],
        [-0.2801, -5.3340, -7.8737],
        [ 0.3246, -1.8925,  3.4005],
        [-0.8731, -2.4596, -2.0578],
        [ 2.5349,  2.0575,  2.3156],
        [-2.6751,  1.9785,  6.3132],
        [ 5.0173, -3.0065, -2.2793],
        [-4.4940,  4.2470, -3.8872],
        [-1.9750, -4.0596,  2.1269],
        [ 1.0775, -5.4534, -4.2132],
        [-3.3446, -0.0206, -2.6886],
        [ 4.5015,  3.1293,  2.3288],
        [-0.6324, -1.3696, -2.3628],
        [ 5.4973, -0.1532,  1.4712],
        [-0.3193, -0.4313, -5.0534],
        [ 2.5220,  1.1990,  3.5328],
        [-2.8744, -2.7022, -1.4893],
        [-1.1798,  2.5833, -1.7099],
        [ 2.2827, -4.2730,  0.2523],
        [-3.2265,  0.0093,  2.5808],
        [-2.6728,  1.9434,  0.2719],
        [ 0.6755,  1.3697, -0.8105],
        [-3.5311,  0.0081,  0.0608],
        [-2.8509, -4.9911,  1.9578],
 

In [26]:
Epot = forces.compute(rot_system, system.box, system.forces, returnDetails=False)
print(Epot)
print(system.forces)

/opt/conda/conda-bld/pytorch_1614378083779/work/aten/src/ATen/native/cuda/IndexKernel.cu:142: operator(): block: [0,0,0], thread: [32,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1614378083779/work/aten/src/ATen/native/cuda/IndexKernel.cu:142: operator(): block: [0,0,0], thread: [3,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1614378083779/work/aten/src/ATen/native/cuda/IndexKernel.cu:142: operator(): block: [0,0,0], thread: [4,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1614378083779/work/aten/src/ATen/native/cuda/IndexKernel.cu:142: operator(): block: [0,0,0], thread: [5,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1614378083779/work/aten/src/ATen/native/cuda/IndexKernel.cu:142: operator(): block

RuntimeError: CUDA error: device-side assert triggered

In [25]:
rot_system = torch.tensor(rot.apply(system.pos[0].cpu())).to(device)